# FLAT input data prepping

This code prepares the final input file to the FLAT model. 

**Original code:** [Konstantinos Pegios](https://github.com/kopegios) <br />
**Conceptualization & Methodological review :** [Alexandros Korkovelos](https://github.com/akorkovelos) & [Konstantinos Pegios](https://github.com/kopegios)<br />
**Updates, Modifications:** [Alexandros Korkovelos](https://github.com/akorkovelos)<br />
**Funding:** The World Bank (contract number: 7190531), [KTH](https://www.kth.se/en/itm/inst/energiteknik/forskning/desa/welcome-to-the-unit-of-energy-systems-analysis-kth-desa-1.197296)

In [35]:
#Import modules and libraries

import geopandas as gpd
import rasterio as rio
import pandas as pd
import fiona
import gdal
import osr
import ogr
import rasterio.mask
import time
import os
import ogr, gdal, osr, os
import numpy as np
import itertools
import re

from osgeo import gdal,ogr
import struct
import csv

import glob

from rasterio.warp import calculate_default_transform, reproject
from rasterio.enums import Resampling
from rasterstats import point_query
from pyproj import Proj
from shapely.geometry import Point, Polygon

In [36]:
# Import data 

path = r"N:\Agrodem\Downscaling\Output_Data\FAO_AgroMap_Crops"
name_of_file = "Moz_Rice_Harv_2000_admin2_1km.csv"

In [37]:
# Import csv as pandas dataframe
df = pd.read_csv(path + "\\" + name_of_file)

In [38]:
#df = df.drop(["yield", "production_ha"], axis=1)
df.rename(columns={'id': 'alloc_key'}, inplace=True)

In [39]:
df.rename(columns={'COUNTRY_COUNTRY': 'state'}, inplace=True)
df.rename(columns={'X_deg': 'lon'}, inplace=True)
df.rename(columns={'Y_deg': 'lat'}, inplace=True)
df.rename(columns={'area_ha': 'statearea_ha'}, inplace=True)
df.rename(columns={'country_code': 'c_code'}, inplace=True)

In [40]:
df.head()

,alloc_key,state,c_code,country,crop,year,harv_area_ha,yield,production_ha,statearea_ha,perimeter_km,lon,lat
0,786895,Ribaue,MOZ,Mozambique,"Rice, Paddy",2000,468.0,0.649573,304.0,644555.756771,406.138864,38.082646,-14.578930
1,786896,Ribaue,MOZ,Mozambique,"Rice, Paddy",2000,468.0,0.649573,304.0,644555.756771,406.138864,38.091929,-14.578966
2,786897,Ribaue,MOZ,Mozambique,"Rice, Paddy",2000,468.0,0.649573,304.0,644555.756771,406.138864,38.101211,-14.579002
3,786941,Ribaue,MOZ,Mozambique,"Rice, Paddy",2000,468.0,0.649573,304.0,644555.756771,406.138864,38.509667,-14.580216
4,786942,Ribaue,MOZ,Mozambique,"Rice, Paddy",2000,468.0,0.649573,304.0,644555.756771,406.138864,38.518951,-14.580235


In [41]:
#Function to change the order of columns in the dataframe 
def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]

In [42]:
# Allocate order in dataframe columns
df = change_column_order(df, 'alloc_key', 1)
df = change_column_order(df, 'lon', 2)
df = change_column_order(df, 'lat', 3)
df = change_column_order(df, 'country', 4)
df = change_column_order(df, 'c_code', 5)
df = change_column_order(df, 'state', 6)
df = change_column_order(df, 'statearea_ha', 7)
df = change_column_order(df, 'perimeter_km', 8)
df = change_column_order(df, 'year', 9)
df = change_column_order(df, 'crop', 10)
df = change_column_order(df, 'harv_area_ha', 11)

In [43]:
df.head()

,alloc_key,lon,lat,country,c_code,state,statearea_ha,perimeter_km,year,crop,yield,harv_area_ha,production_ha
0,786895,38.082646,-14.578930,Mozambique,MOZ,Ribaue,644555.756771,406.138864,2000,"Rice, Paddy",0.649573,468.0,304.0
1,786896,38.091929,-14.578966,Mozambique,MOZ,Ribaue,644555.756771,406.138864,2000,"Rice, Paddy",0.649573,468.0,304.0
2,786897,38.101211,-14.579002,Mozambique,MOZ,Ribaue,644555.756771,406.138864,2000,"Rice, Paddy",0.649573,468.0,304.0
3,786941,38.509667,-14.580216,Mozambique,MOZ,Ribaue,644555.756771,406.138864,2000,"Rice, Paddy",0.649573,468.0,304.0
4,786942,38.518951,-14.580235,Mozambique,MOZ,Ribaue,644555.756771,406.138864,2000,"Rice, Paddy",0.649573,468.0,304.0


In [44]:
# Add geometry and convert to spatial dataframe in source CRS
df['geometry'] = list(zip(df['lon'], df['lat']))
df['geometry'] = df['geometry'].apply(Point)
df = gpd.GeoDataFrame(df, geometry='geometry', crs={'init': 'epsg:4326'})

# Reproject data in to Ordnance Survey GB coordinates
df_osgb = df.to_crs({'init': 'epsg:32737'})

In [45]:
df_osgb.head()

,alloc_key,lon,lat,country,c_code,state,statearea_ha,perimeter_km,year,crop,yield,harv_area_ha,production_ha,geometry
0,786895,38.082646,-14.578930,Mozambique,MOZ,Ribaue,644555.756771,406.138864,2000,"Rice, Paddy",0.649573,468.0,304.0,POINT (401183.558 8388046.256)
1,786896,38.091929,-14.578966,Mozambique,MOZ,Ribaue,644555.756771,406.138864,2000,"Rice, Paddy",0.649573,468.0,304.0,POINT (402183.558 8388046.256)
2,786897,38.101211,-14.579002,Mozambique,MOZ,Ribaue,644555.756771,406.138864,2000,"Rice, Paddy",0.649573,468.0,304.0,POINT (403183.558 8388046.256)
3,786941,38.509667,-14.580216,Mozambique,MOZ,Ribaue,644555.756771,406.138864,2000,"Rice, Paddy",0.649573,468.0,304.0,POINT (447183.558 8388046.256)
4,786942,38.518951,-14.580235,Mozambique,MOZ,Ribaue,644555.756771,406.138864,2000,"Rice, Paddy",0.649573,468.0,304.0,POINT (448183.558 8388046.256)


<div class="alert alert-block alert-warning">
<b>Note:</b> The geodataframe, contains crop points with needed attributes including a geometry column. It needs to be exported as shapefile. However, when this is done with geopandas, the column geometry (Point(x,y)) is not transfered in the attribute table. This is however needed for the extraction process below, otherwise "None" values are returned. In order to overcome this issue, the csv is imported into Qgis and then exported as feature layer in the same folder. This solves the issue. 
</div>

In [46]:
# Define output path
# Overwriting the csv file
path = r"N:\Agrodem\Downscaling\Output_Data\FAO_AgroMap_Crops"
shpname = "Moz_Rice_Harv_2000_admin2_1km.csv"

#drybeans
df_osgb.to_csv(os.path.join(path,"{c}".format(c=shpname)), index=False)

#### >>> Please open Qgis and act as suggested above<<<

## Extracting raster input to csv 

In [47]:
raster_path = r"N:\Agrodem\Downscaling\Input_Data\rasters"
raster_files =[]
csvoutpath = r"N:\Agrodem\Downscaling\Output_Data\FLAT_input\1km_Rice"

print ("Reading independent variables...")

for i in os.listdir(raster_path):
    if i.endswith('.tif'):
        raster_files.append(i)  

shp_filename = r"N:\Agrodem\Downscaling\Output_Data\FAO_AgroMap_Crops\shapefiles\Moz_Rice_Harv_2000_admin2_1km.shp"

print ("Extracting raster values to points...")

for i in raster_files:
    print("Extracting " + i + " values...")
    src_filename = raster_path + "\\" + i 
    li_values = list()

    src_ds=gdal.Open(src_filename) 
    gt=src_ds.GetGeoTransform()
    rb=src_ds.GetRasterBand(1)
    ds=ogr.Open(shp_filename)
    lyr=ds.GetLayer()
    
    for feat in lyr:
        geom = feat.GetGeometryRef()
        feat_id = feat.GetField('alloc_key')
        mx,my=geom.GetX(), geom.GetY()  #coord in map units

        #Convert from map to pixel coordinates.
        #Only works for geotransforms with no rotation.
        px = int((mx - gt[0]) / gt[1]) #x pixel
        py = int((my - gt[3]) / gt[5]) #y pixel

        intval=rb.ReadAsArray(px,py,1,1)
        li_values.append([feat_id, intval[0]])
        
    print ("Writing " + i + " values to csv...")
    
    with open(csvoutpath + "\\" + i.split('.')[0] + '.csv', 'w') as csvfile:
        wr = csv.writer(csvfile)
        wr.writerows(li_values) 

Reading independent variables...
Extracting raster values to points...
Extracting BDRICM.tif values...
Writing BDRICM.tif values to csv...
Extracting BLD.tif values...
Writing BLD.tif values to csv...
Extracting CLYPPT.tif values...
Writing CLYPPT.tif values to csv...
Extracting DRAINFAO.tif values...
Writing DRAINFAO.tif values to csv...
Extracting elevation.tif values...
Writing elevation.tif values to csv...
Extracting EVI.tif values...
Writing EVI.tif values to csv...
Extracting Landcover30m.tif values...
Writing Landcover30m.tif values to csv...
Extracting NDVI.tif values...
Writing NDVI.tif values to csv...
Extracting ORCDRC.tif values...
Writing ORCDRC.tif values to csv...
Extracting PHIHOX.tif values...
Writing PHIHOX.tif values to csv...
Extracting prec.tif values...
Writing prec.tif values to csv...
Extracting slope.tif values...
Writing slope.tif values to csv...
Extracting srad.tif values...
Writing srad.tif values to csv...
Extracting tavg.tif values...
Writing tavg.tif va

## Merge csv files with crop

In [48]:
#Import data 

path = r"N:\Agrodem\Downscaling\Output_Data\FAO_AgroMap_Crops"
name_of_file = "Moz_Rice_Harv_2000_admin2_1km.csv"

flat_input = pd.read_csv(path + "\\" + name_of_file)

In [49]:
csvoutpath = r"N:\Agrodem\Downscaling\Output_Data\FLAT_input\1km_Rice"

csv_files = []

print ("Reading csv files...")

for i in os.listdir(csvoutpath):
    if i.endswith('.csv'):
        csv_files.append(i)  

for i in csv_files:
    print('Reading...'+ i)    
    df_csv = pd.read_csv(csvoutpath + "//" + i, index_col=None, header=None)
    df_csv.iloc[:,1] = df_csv.iloc[:,1].astype(str)
    df_csv.iloc[:,1] = df_csv.iloc[:,1].str.replace('[','')
    df_csv.iloc[:,1] = df_csv.iloc[:,1].str.replace(']','')
    columnName = i.split('.')[0]
    print("Merging..." + columnName)
    flat_input[columnName] = df_csv.iloc[:,1]

Reading csv files...
Reading...BDRICM.csv
Merging...BDRICM
Reading...BLD.csv
Merging...BLD
Reading...CLYPPT.csv
Merging...CLYPPT
Reading...DRAINFAO.csv
Merging...DRAINFAO
Reading...elevation.csv
Merging...elevation
Reading...EVI.csv
Merging...EVI
Reading...Landcover30m.csv
Merging...Landcover30m
Reading...NDVI.csv
Merging...NDVI
Reading...ORCDRC.csv
Merging...ORCDRC
Reading...PHIHOX.csv
Merging...PHIHOX
Reading...prec.csv
Merging...prec
Reading...slope.csv
Merging...slope
Reading...srad.csv
Merging...srad
Reading...tavg.csv
Merging...tavg
Reading...TEXMHT.csv
Merging...TEXMHT
Reading...wind.csv
Merging...wind


## Cleaning and normalizing dataframe

In [50]:
#Show columns
flat_input.columns

Index(['alloc_key', 'lon', 'lat', 'country', 'c_code', 'state', 'statearea_ha',
       'perimeter_km', 'year', 'crop', 'yield', 'harv_area_ha',
       'production_ha', 'geometry', 'BDRICM', 'BLD', 'CLYPPT', 'DRAINFAO',
       'elevation', 'EVI', 'Landcover30m', 'NDVI', 'ORCDRC', 'PHIHOX', 'prec',
       'slope', 'srad', 'tavg', 'TEXMHT', 'wind'],
      dtype='object')

In [51]:
#Delete geometry column
flat_input = flat_input.drop(["geometry"], axis=1)

#### Convert values in newly added columns into float 

In [52]:
flat_input["harv_area_ha"] = flat_input["harv_area_ha"].astype(float)
flat_input["BDRICM"] = flat_input["BDRICM"].astype(float)
flat_input["BLD"] = flat_input["BLD"].astype(float)
flat_input["CLYPPT"] = flat_input["CLYPPT"].astype(float)
flat_input["DRAINFAO"] = flat_input["DRAINFAO"].astype(float)
flat_input["elevation"] = flat_input["elevation"].astype(float)
flat_input["EVI"] = flat_input["EVI"].astype(float)
flat_input["Landcover30m"] = flat_input["Landcover30m"].astype(float)
flat_input["NDVI"] = flat_input["NDVI"].astype(float)
flat_input["ORCDRC"] = flat_input["ORCDRC"].astype(float)
flat_input["PHIHOX"] = flat_input["PHIHOX"].astype(float)
flat_input["prec"] = flat_input["prec"].astype(float)
flat_input["slope"] = flat_input["slope"].astype(float)
flat_input["srad"] = flat_input["srad"].astype(float)
flat_input["tavg"] = flat_input["tavg"].astype(float)
flat_input["TEXMHT"] = flat_input["TEXMHT"].astype(float)
flat_input["wind"] = flat_input["wind"].astype(float)

In [53]:
flat_input.dtypes

alloc_key          int64
lon              float64
lat              float64
country           object
c_code            object
state             object
statearea_ha     float64
perimeter_km     float64
year               int64
crop              object
yield            float64
harv_area_ha     float64
production_ha    float64
BDRICM           float64
BLD              float64
CLYPPT           float64
DRAINFAO         float64
elevation        float64
EVI              float64
Landcover30m     float64
NDVI             float64
ORCDRC           float64
PHIHOX           float64
prec             float64
slope            float64
srad             float64
tavg             float64
TEXMHT           float64
wind             float64
dtype: object

#### Normalize EVI and NDVI 

In [54]:
flat_input['EVI'] = flat_input['EVI']*0.0001
flat_input['NDVI'] = flat_input['NDVI']*0.0001

#### Keep areas that are indicated as cropland in GFSAD30m

In [55]:
flat_input = flat_input[flat_input["Landcover30m"] == 2]
flat_input = flat_input.drop(["Landcover30m"], axis=1)

#### Fixing out-of-range values

In [56]:
# remove blank spaces in state names    
flat_input["state"].replace('\s+', '_',regex=True,inplace=True)
flat_input["state"].replace("'", '_',regex=True,inplace=True)

In [57]:
# Re-indexing allocation keys to avoid dublicates
flat_input = flat_input.assign(alloc_key=np.arange(len(flat_input))).reset_index(drop=True)

In [58]:
# Turning NaN rows to 0
flat_input.fillna(0,inplace=True)

In [59]:
# Turn all negative values to 0
pred_columns = ['tavg', 'srad', 'prec', 'wind', 'PHIHOX',
                'BDRICM', 'BLD', 'CLYPPT', 'TEXMHT', 'ORCDRC',
                'DRAINFAO', 'slope', 'elevation']

for col in pred_columns:
    mvalue = flat_input[col].mean()
    print (mvalue)
    flat_input[col][flat_input[col]<0] = 0

-inf
17293.881901764777
94.85783041953599
-2.0979883993582622e+34
60.66296433419721
135.74642107861285
1356.2374429223744
22.250462791558682
7.526780204862397
-0.13078489448352462
4.802542268295693
1.0964141028872023
407.91098975688016


C:\Users\adm.esa\Anaconda3\envs\testgdal\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [60]:
for col in pred_columns:
    mvalue = flat_input[col].mean()
    print (mvalue)

23.602824565222758
17293.881901764777
96.87979759218808
1.4838732568184625
60.66296433419721
135.74642107861285
1371.0452918672097
22.250462791558682
7.526780204862397
14.677064050351722
4.802542268295693
1.0964141028872023
407.91098975688016


In [61]:
flat_input.columns

Index(['alloc_key', 'lon', 'lat', 'country', 'c_code', 'state', 'statearea_ha',
       'perimeter_km', 'year', 'crop', 'yield', 'harv_area_ha',
       'production_ha', 'BDRICM', 'BLD', 'CLYPPT', 'DRAINFAO', 'elevation',
       'EVI', 'NDVI', 'ORCDRC', 'PHIHOX', 'prec', 'slope', 'srad', 'tavg',
       'TEXMHT', 'wind'],
      dtype='object')

In [62]:
flat_input = change_column_order(flat_input, 'alloc_key', 0)
flat_input = change_column_order(flat_input, 'state', 1)
flat_input = change_column_order(flat_input, 'lon', 2)
flat_input = change_column_order(flat_input, 'lat', 3)
flat_input = change_column_order(flat_input, 'country', 4)
flat_input = change_column_order(flat_input, 'c_code', 5)

flat_input = change_column_order(flat_input, 'statearea_ha', 6)
flat_input = change_column_order(flat_input, 'perimeter_km', 7)
flat_input = change_column_order(flat_input, 'year', 8)
flat_input = change_column_order(flat_input, 'crop', 9)
flat_input = change_column_order(flat_input, 'harv_area_ha', 10)
flat_input = change_column_order(flat_input, 'yield', 11)
flat_input = change_column_order(flat_input, 'production_ha', 12)

flat_input = change_column_order(flat_input, 'tavg', 13)
flat_input = change_column_order(flat_input, 'srad', 14)
flat_input = change_column_order(flat_input, 'prec', 15)
flat_input = change_column_order(flat_input, 'wind', 16)

flat_input = change_column_order(flat_input, 'PHIHOX', 17)
flat_input = change_column_order(flat_input, 'BDRICM', 18)
flat_input = change_column_order(flat_input, 'BLD', 19)
flat_input = change_column_order(flat_input, 'CLYPPT', 20)
flat_input = change_column_order(flat_input, 'TEXMHT', 21)
flat_input = change_column_order(flat_input, 'ORCDRC', 22)
flat_input = change_column_order(flat_input, 'DRAINFAO', 23)

flat_input = change_column_order(flat_input, 'EVI', 24)
flat_input = change_column_order(flat_input, 'NDVI', 25)

flat_input = change_column_order(flat_input, 'slope', 26)
flat_input = change_column_order(flat_input, 'elevation', 27)

In [63]:
flat_input.head()

,alloc_key,state,lon,lat,country,c_code,statearea_ha,perimeter_km,year,crop,...,BDRICM,BLD,CLYPPT,TEXMHT,ORCDRC,DRAINFAO,EVI,NDVI,slope,elevation
0,0,Ribaue,38.203257,-14.597454,Mozambique,MOZ,644555.756771,406.138864,2000,"Rice, Paddy",...,99.0,1340.0,25.0,6.0,13.0,5.0,0.3492,0.5918,0.881128,455.0
1,1,Ribaue,38.305378,-14.597780,Mozambique,MOZ,644555.756771,406.138864,2000,"Rice, Paddy",...,136.0,1370.0,23.0,6.0,16.0,5.0,0.3446,0.6086,1.033320,460.0
2,2,Ribaue,38.166054,-14.615405,Mozambique,MOZ,644555.756771,406.138864,2000,"Rice, Paddy",...,105.0,1380.0,21.0,6.0,14.0,5.0,0.3574,0.6339,0.730796,491.0
3,3,Ribaue,38.249614,-14.615689,Mozambique,MOZ,644555.756771,406.138864,2000,"Rice, Paddy",...,120.0,1380.0,21.0,6.0,13.0,5.0,0.3609,0.6420,0.638606,463.0
4,4,Ribaue,38.416738,-14.616167,Mozambique,MOZ,644555.756771,406.138864,2000,"Rice, Paddy",...,96.0,1400.0,22.0,6.0,14.0,5.0,0.3140,0.5932,1.055811,445.0


In [64]:
flat_input.columns

Index(['alloc_key', 'state', 'lon', 'lat', 'country', 'c_code', 'statearea_ha',
       'perimeter_km', 'year', 'crop', 'harv_area_ha', 'yield',
       'production_ha', 'tavg', 'srad', 'prec', 'wind', 'PHIHOX', 'BDRICM',
       'BLD', 'CLYPPT', 'TEXMHT', 'ORCDRC', 'DRAINFAO', 'EVI', 'NDVI', 'slope',
       'elevation'],
      dtype='object')

#### Final column fixing

In [65]:
# give crop name
crop_modelled ="Rice"

# Dropping columns
flat_input = flat_input.drop(["country"], axis=1)
flat_input = flat_input.drop(["c_code"], axis=1)
flat_input = flat_input.drop(["perimeter_km"], axis=1)
flat_input = flat_input.drop(["year"], axis=1)
flat_input = flat_input.drop(["crop"], axis=1)
flat_input = flat_input.drop(["yield"], axis=1)
flat_input = flat_input.drop(["production_ha"], axis=1)


#Remaning columns
flat_input.rename(columns={'state': 'NAME'}, inplace=True)
flat_input.rename(columns={'statearea_ha': 'statearea'}, inplace=True)
flat_input.rename(columns={'harv_area_ha': crop_modelled}, inplace=True)


In [66]:
flat_input

,alloc_key,NAME,lon,lat,statearea,Rice,tavg,srad,prec,wind,...,BDRICM,BLD,CLYPPT,TEXMHT,ORCDRC,DRAINFAO,EVI,NDVI,slope,elevation
0,0,Ribaue,38.203257,-14.597454,644555.756771,468.0,24.166666,17118.084,86.583336,1.2,...,99.0,1340.0,25.0,6.0,13.0,5.0,0.3492,0.5918,0.881128,455.0
1,1,Ribaue,38.305378,-14.597780,644555.756771,468.0,24.183334,17175.166,87.583336,1.3,...,136.0,1370.0,23.0,6.0,16.0,5.0,0.3446,0.6086,1.033320,460.0
2,2,Ribaue,38.166054,-14.615405,644555.756771,468.0,23.925001,17078.916,87.083336,1.2,...,105.0,1380.0,21.0,6.0,14.0,5.0,0.3574,0.6339,0.730796,491.0
3,3,Ribaue,38.249614,-14.615689,644555.756771,468.0,24.233332,17113.000,87.333336,1.2,...,120.0,1380.0,21.0,6.0,13.0,5.0,0.3609,0.6420,0.638606,463.0
4,4,Ribaue,38.416738,-14.616167,644555.756771,468.0,24.258331,17278.166,87.916664,1.3,...,96.0,1400.0,22.0,6.0,14.0,5.0,0.3140,0.5932,1.055811,445.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32407,32407,Chinde,36.673672,-18.363305,403497.970729,8487.0,24.966667,17933.416,96.666664,3.0,...,159.0,1440.0,20.0,9.0,12.0,3.0,0.3424,0.6176,0.122152,10.0
32408,32408,Chinde,36.433483,-18.613083,403497.970729,8487.0,25.266668,17777.250,93.166664,2.9,...,153.0,1420.0,22.0,6.0,13.0,3.0,0.3954,0.6163,0.209732,11.0
32409,32409,Chinde,36.442953,-18.613212,403497.970729,8487.0,25.233332,17790.666,92.916664,3.0,...,141.0,1410.0,30.0,6.0,20.0,2.0,0.3342,0.5587,0.289405,9.0
32410,32410,Chinde,36.395191,-18.639649,403497.970729,8487.0,25.308332,17719.084,94.833336,2.6,...,165.0,1420.0,31.0,6.0,14.0,2.0,0.3516,0.6226,0.364769,4.0


In [67]:
# This part prints full results

path = r"N:\Agrodem\Downscaling\Output_Data\FLAT_input"
name_of_flat_input_file = "flat_input_Rice_1km"

flat_input.to_csv(os.path.join(path,"{c}.csv".format(c=name_of_flat_input_file)), index=False)